Now we have 14 model

For this notebook we will put our picture through ALL the 14 model

and we will then take the three highest (1's) valued models

    by having a 2Darray that hold 14 models x [0 , 1] (softmax) 

function1(){
    return arrayofVal.appended
}

function2(){
    takes poster and turns it into np.array
}

function3(){
    takes arrayOfVals
    returns best three genres with their percentages
}

main(){
input = poster

function2(poster)
arrayyofVals = []
arrayofVals.append (function1(poster, comedy))
function1(poster, action)
function1(poster, horror)
function1(poster, ...)
function1(poster, ...)
function1(poster, ...)
function1(poster, 14...)


function3()


done :)


}


In [1]:
from PIL import Image
import pandas as pd
import numpy as np
from keras.layers import Input, Dense, Conv2D, MaxPooling2D, UpSampling2D, Flatten, Dropout
from keras.models import Sequential
from keras.datasets import mnist
from tensorflow.keras.utils import to_categorical
import numpy as np
import matplotlib.pyplot as plt
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
import os
import tensorflow as tf
from tensorflow import keras

In [2]:
def PosterToArray(image):
    poster = np.array(image)
    poster = poster.astype('float32') / 255
    return poster

In [3]:
def PredictByModel(ourGenre, posterPath):

    poster = Image.open(posterPath)
    poster = poster.resize((100,100))
    numPyPoster = PosterToArray(poster)
    posterArray = [numPyPoster]
    posterArray = np.array(posterArray) 
    checkpoint_path = f"Weights/{ourGenre}/cp.ckpt"

    model = Sequential()
    model.add(Conv2D(64, (3, 3), activation='relu', padding='same', input_shape=(100, 100, 3)))
    model.add(MaxPooling2D((2, 2), padding='same'))
    model.add(Conv2D(32, (3, 3), activation='relu', padding='same'))
    model.add(MaxPooling2D((2, 2), padding='same'))
    model.add(Flatten())
    model.add(Dropout(0.2))
    model.add(Dense(2, activation="softmax"))
    model.compile(optimizer='adam', loss='CategoricalCrossentropy', metrics=["accuracy"])

    model.load_weights(checkpoint_path).expect_partial()
    
    predicted = model.predict(posterArray)
    return (predicted[0], ourGenre)
    
    

In [4]:
def PredictAllModes(posterPath, allPredictions):
    Genres = ['Action', 'Adventure', 'Biography', 'Comedy', 'Crime', 'Documentary', 'Drama', 'Family', 'Fantasy', 'Horror', 'Music', 'Mystery', 'Romance', 'Sci-Fi', 'Thriller']
    for i in range(len(Genres)):
        allPredictions.append(PredictByModel(Genres[i], posterPath, allPredictions))
    return allPredictions

In [5]:
def ThreeGenres(allPredictions):
    tempSort = []
    for i in range(len(allPredictions)):
        temp = [allPredictions[i][0][1] , allPredictions[i][1]]
        tempSort.append(temp)
    tempSort.sort()
    tempSort.reverse()
    print(f"Genre1: {tempSort[0][1]} ({np.round((tempSort[0][0] * 100),2)}%)    Genre2: {tempSort[1][1]} ({np.round(tempSort[1][0] * 100,2)}%)    Genre3: {tempSort[2][1]}({np.round(tempSort[2][0] * 100,2)}%)")
    
    

In [6]:
Genres = ['Action', 'Adventure', 'Biography', 'Comedy', 'Crime', 'Documentary', 'Drama', 'Family', 'Fantasy', 'Horror', 'Music', 'Mystery', 'Romance', 'Sci-Fi', 'Thriller']
posterPath = f"test6.jpg"
allPredictions = []
for i in range(len(Genres)):
    allPredictions.append(PredictByModel(Genres[i], posterPath))
allPredictions

2022-01-07 14:14:43.288271: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


[(array([0.3480003, 0.6519998], dtype=float32), 'Action'),
 (array([0.1012907 , 0.89870924], dtype=float32), 'Adventure'),
 (array([0.09391142, 0.9060886 ], dtype=float32), 'Biography'),
 (array([0.14327492, 0.85672504], dtype=float32), 'Comedy'),
 (array([0.31716082, 0.6828392 ], dtype=float32), 'Crime'),
 (array([0.02813013, 0.9718698 ], dtype=float32), 'Documentary'),
 (array([0.06563267, 0.93436736], dtype=float32), 'Drama'),
 (array([0.26021093, 0.73978907], dtype=float32), 'Family'),
 (array([0.20410341, 0.79589653], dtype=float32), 'Fantasy'),
 (array([0.79623663, 0.20376335], dtype=float32), 'Horror'),
 (array([0.4233404, 0.5766597], dtype=float32), 'Music'),
 (array([0.6789192 , 0.32108083], dtype=float32), 'Mystery'),
 (array([0.10365506, 0.8963449 ], dtype=float32), 'Romance'),
 (array([0.26879928, 0.7312007 ], dtype=float32), 'Sci-Fi'),
 (array([0.5137334 , 0.48626658], dtype=float32), 'Thriller')]

In [7]:
ThreeGenres(allPredictions)

Genre1: Documentary (97.19%)    Genre2: Drama (93.44%)    Genre3: Biography(90.61%)
